In [1]:
from utils.dataset import set_b_dataset, set_b_dataclass
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.models as models
from random import sample

PROJECT_DIR = '/Users/nattapolchanpaisit/Documents/GitHub/Algorithm/SC1015-PROJECT/'
N_TRAIN = 300

train_id = tuple(sample([i for i in range(461)], N_TRAIN))

dataset = set_b_dataclass(PROJECT_DIR, output_width=200, list_id=train_id)

# Writer will output to ./runs/ directory by default
# writer = SummaryWriter()

# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# trainset = datasets.MNIST('mnist_train', train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
# model = torchvision.models.resnet50(False)
# # Have ResNet model take in grayscale rather than RGB
# model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# images, labels = next(iter(trainloader))

# grid = torchvision.utils.make_grid(images)
# writer.add_image('images', grid, 0)
# writer.add_graph(model, images)
# writer.close()

/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <BE0CCD9A-269A-30E2-A23C-DA45E89EBB1F> /Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/nattapolchanpaisit/miniforge3/lib/python3.10/lib-dynload/../../libjpeg.8.dylib' (no such file), '/Users/nattapolchanpaisit/miniforge3/bin/../lib/libjpeg.8.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS@rpath/libjpeg.8.dylib' (no such file), '/Users/nattapolchanpaisit/miniforge3/li

In [3]:
from torch.utils.data import DataLoader

writer = SummaryWriter()
train_id = tuple(sample([i for i in range(461)], N_TRAIN))
trainset = set_b_dataclass(PROJECT_DIR, output_width=200, list_id=train_id)
trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
model = torchvision.models.resnet18(False)

for epoch, data in enumerate(trainloader):
    stft, label = data
    print(epoch)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
